# Podstawy podejmowania decyzji projekt
283018

## Opis

1. Plansza (macierz) nxn zapełniona symbolami.
2. Każda komórka macierzy zawiera jeden symbol w formacie heksadecymalnym (nie ma to zasadniczego znaczenia i można je zamienić na liczby całkowite lub symbole ascii) z pewnej ograniczonej puli.
3. Symbole w różnych komórkach macierzy mogą się powtarzać.
4. Jest lista "demonów" - sekwencji różnej długości złożonych z symboli z tej samej puli (przy czym jedna sekwencja może zawierać inną sekwencję w części albo w pełni).
5. Jest ograniczony "buffor" - maksymalna ilość symboli które można aktywować/wybrać na macierzy.
6. Wybierać symbole z planszy można według określonych zasad:<br>
	a) w pierwszym kroku można wybrać dowolny symbol z pierwszego *wiersza*, na przykład (0, 4)<br>
	b) w drugim kroku symbol z tej samej *kolumny* co poprzednio wybrany symbol, np. (3, 4)<br>
	c) w trzecim znowu z tego samego *wiersza* co poprzednio wybrany, np. (5, 4)<br>
	Na ogół, jeśli krok jest nieparzysty to wybór się odbywa w odpowiednim (zgodnym z poprzednio wybranym) *wierszu*, jeśli parzysty to w *kolumnie*.
7. Można wybierać symbol z tej samej komórki macierzy tylko jednokrotnie
8. Jeśli zostały kolejno wybrane symbole odpowiadające "demonowi" to ta sekwencja jest "aktywowana".
9. Można wybierać symbole niebędące elementem żadnego "demona".
10. Macierz jest generowana w taki sposób by każdy pojedynczy "demon" był możliwy do aktywacji

# Instalacja

In [ ]:
!pip install gurobipy
!pip install icecream


In [1]:
import gurobipy as gp
import numpy as np  # noqa: F401
from icecream import ic  # noqa: F401
# import matplotlib.pyplot as plt

# ------

In [6]:
# dane wejściowe
buffer_size = 8
demons = [
    [1, 3, 4, 1],
    [3, 1, 2],
    [1, 3, 1, 2, 4],
]
d_costs = [3, 2, 5]
matrix = [
    [1, 3, 3],
    [4, 2, 1],
    [3, 1, 4]
]


# aktualna konfiguracja (stałe)
d_amo = len(demons)
d_lengths = [len(i) for i in demons]
matrix_size = len(matrix)
if not all(len(row) == matrix_size for row in matrix):
    raise ValueError("Matrix must have same dimensions")



model = gp.Model("Breach Protocol")
model.setParam("OutputFlag", 0)



# wybór drogi
x = model.addVars(matrix_size, matrix_size, buffer_size, vtype=gp.GRB.BINARY, name='x')

# tylko jedna komórka wybierana w każdym kroku
for t in range(buffer_size):
    model.addConstr(x.sum('*', '*', t) == 1, name=f'one_cell_per_step_{t}')
#TODO: brak wyboru dostępny? (jeśli tak to może być tylko na końcu)

# maksymalna ilość kroków (prawdopodobnie zbędne)
model.addConstr((x.sum('*', '*', '*') <= buffer_size), name="max_steps")



# wybrane demony
y = model.addVars(d_amo, vtype=gp.GRB.BINARY, name='y')

# !TODO: jeden demon może się zawierać w drugim
# długość wszystkich nie przekracza buffor
model.addConstr(gp.quicksum(d_lengths[i] * y[i] for i in range(d_amo)) <= buffer_size, name='buffer_len_constr')





# maksymalizacja punktów (obv)
model.setObjective(gp.quicksum(d_costs[i] * y[i] for i in range(d_amo)), gp.GRB.MAXIMIZE)
model.optimize()



print("\n"+"#"*50)
if model.status == gp.GRB.OPTIMAL:
    sol = 0
    print("Selected")
    for i in range(len(y)):
        if y[i].x > 0.5:
            print(f"{i + 1}  (length: {d_lengths[i]}, cost: {d_costs[i]})")
            sol += d_costs[i]
    print(f"Buffer: {sol}/{buffer_size}")
else:
    print("No solution found")



##################################################
Selected
2  (length: 3, cost: 2)
3  (length: 5, cost: 5)
Buffer: 7/8
